In [1]:
%load_ext autoreload
%autoreload 2

from rerun_viz.living_room import setup_blueprint
from utils.data_loading import load_all_color_images, load_all_depth_images
import rerun as rr

In [3]:
rgb = load_all_color_images()
depth = load_all_depth_images()


In [4]:
import rerun as rr

rr.init("living_room", spawn=True)

for img in rgb[:10]:
    rr.log("world/camera/image/rgb", rr.Image(img, color_model=rr.ColorModel.RGB))

for d in depth[:10]:
    rr.log("world/camera/image/depth", rr.DepthImage(d))

[2025-07-08T08:15:50Z INFO  re_grpc_server] Listening for gRPC connections on 0.0.0.0:9876. Connect by running `rerun --connect rerun+http://127.0.0.1:9876/proxy`


[2025-07-08T08:17:12Z ERROR re_grpc_client::message_proxy::write] Write messages call failed: status: Unknown, message: "transport error", details: [], metadata: MetadataMap { headers: {} }
